In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import math
from scipy import stats
from scipy.integrate import quad
from scipy.stats import norm

# Case 6: LIBOR Market Model

## Part 1: Discount Bond Prices

In [4]:
sigma = 0.01  
numSteps = 10  
numSimulations = 1000
dT = 1
f0 = 0.03
N = 10

In [5]:
# Exercise 1.1
maturities = np.arange(1, N + 1)
discountBond = [1 / (1 + dT*f0)**m for m in maturities]

# Create DataFrame
discountBond_df = pd.DataFrame({
    "T": maturities,
    "Discount Bond Price": discountBond
})
pd.set_option("display.precision", 15)

discountBond_df.head(10)

,T,Discount Bond Price
0,1,0.970873786407767
1,2,0.942595909133754
2,3,0.915141659353160
3,4,0.888487047915689
4,5,0.862608784384164
5,6,0.837484256683654
6,7,0.813091511343354
7,8,0.789409234313936
8,9,0.766416732343627
9,10,0.744093914896725


In [6]:
def discountBond_prices(f0, dT, N):
    return np.array([(1 / (1 + dT*f0)) ** i for i in range(1, N + 1)])

print(discountBond_prices(f0 = 0.03, dT=1, N=10))

[0.97087379 0.94259591 0.91514166 0.88848705 0.86260878 0.83748426
 0.81309151 0.78940923 0.76641673 0.74409391]


In [ ]:

# Parameters
N = 10  # Number of periods
delta_T = 1  # Time step size
f_0 = 0.03  # Initial forward rate (3%)
sigma = 0.01  # Volatility (1%)
num_paths = 1000000  # Number of Monte Carlo paths
num_steps = 10  # Number of time steps

# Question 1: Compute discount bond prices D(0, T_i)
D0_T = []  # List to store D(0, T_i)
for i in range(1, N):
    D_i = (1 / (1 + f_0)) ** i  # Discount bond price formula
    D0_T.append(D_i)
    print(f"D(0, T_{i}) = {D_i:.19f}")
    

# Include D(0, T_N) in the list
D0_T = np.array(D0_T + [(1 / (1 + f_0)) ** N])

print("\nDiscount bond prices at t=0:")
for i in range(1, N + 1):
    print(f"D(0, T_{i}) = {D0_T[i - 1]:.19f}")

# Question 2: Monte Carlo simulation for forward LIBOR rates
f = np.full((num_paths, N), f_0)

# Simulate forward rates under terminal measure Q^10
for step in range(1, num_steps + 1):
    dt = delta_T / num_steps  # Time increment
    dW = np.random.normal(0, np.sqrt(dt), (num_paths, N))  # Brownian increments
    
    for i in range(N - 1):
        # Calculate the drift term for f_i under Q^10
        drift = np.zeros(num_paths)
        for k in range(i + 1, N):
            drift -= (delta_T * sigma * f[:, k]) / (1 + delta_T * f[:, k])
        
        # Update forward rates using SDE
        f[:, i] += drift * dt + sigma * dW[:, i]

print("\nMonte Carlo simulation of forward rates completed.")

# Question 3: Compute Monte Carlo discount bond prices
D_MC = np.ones((num_paths, N))  
for i in range(N):
    D_MC[:, i] = np.cumprod(1 / (1 + delta_T * f[:, :i + 1]), axis=1)[:, -1]

D_MC_avg = np.mean(D_MC, axis=0)
D_MC_std = np.std(D_MC, axis=0)
D_MC_se = D_MC_std / np.sqrt(num_paths)

print("\nMonte Carlo discount bond prices and standard errors:")
for i, (price, se) in enumerate(zip(D_MC_avg, D_MC_se), 1):
    print(f"D_MC({i}) = {price:.19f} ± {se:.19f}")

print("\nComparison with initial term structure:")
for i in range(N):
    print(f"T{i+1}: Initial = {D0_T[i]:.19f}, MC = {D_MC_avg[i]:.19f} ± {D_MC_se[i]:.19f}, "
          f"Diff = {D_MC_avg[i] - D0_T[i]:.19f}")

## Part 2: Gaussian swaption formulas

In [ ]:
f0 = 0.03         
sigma = 0.01       
N = 10             
dT = 1        
strikes = np.array([0.01, 0.02, 0.03, 0.04, 0.05])  
num_simulations = 100000  

def discountBond_prices(f0, N, dT):
    maturities = np.arange(1, N + 1) * dT
    return (1 / (1 + f0)) ** maturities

def parSwapRates(D, N, dT):
    par_swap_rates = []
    annuities = []

    for i in range(1, N):
        sum_D = np.sum(D[i:]) 
        y_Ti_T10 = (D[i - 1] - D[-1]) / sum_D

        par_swap_rates.append(y_Ti_T10)
        annuities.append(sum_D)

    return np.array(par_swap_rates), np.array(annuities)

def gaussianSwaption_prices(par_swap_rates, annuities, strikes, sigma, N, dT):
    option_maturities = np.arange(1, N) * dT
    swaption_data = []

    for maturity_index, maturity in enumerate(option_maturities):
        annuity = annuities[maturity_index]
        par_swap_rate = par_swap_rates[maturity_index]
        
        for strike in strikes:
            # Calculate Gaussian parameters
            d = (par_swap_rate - strike) / (sigma * np.sqrt(maturity))
            
            # Gaussian swaption price
            swaption_price = annuity * (
                (par_swap_rate - strike) * norm.cdf(d) 
                + sigma * np.sqrt(maturity) * norm.pdf(d)
            )
            
            swaption_data.append({
                'Option Maturity (Years)': maturity,
                'Strike (%)': strike * 100,
                'Gaussian Swaption Price': swaption_price
            })

    return pd.DataFrame(swaption_data)
   
D = discountBond_prices(f0, N, dT)
par_swap_rates, annuities = parSwapRates(D, N, dT)
gaussian_prices_df = gaussianSwaption_prices(par_swap_rates, annuities, strikes, sigma, N, dT)
gaussian_prices_df.head(100)



,Option Maturity (Years),Strike (%),Gaussian Swaption Price
0,1,1.0,0.151828
1,1,2.0,0.081891
2,1,3.0,0.030157
3,1,4.0,0.006298
4,1,5.0,0.000642
5,2,1.0,0.135660
6,2,2.0,0.079377
7,2,3.0,0.037331
8,2,4.0,0.013210
9,2,5.0,0.003325


In [26]:
def simulatedParams(f_t_num, T_i, T):

    discountBond = np.array([
        1 / ((1 + f_t_num[:, i]) ** (i + 1)) for i in range(T)
    ]).T

    par_swap_rate = (
        discountBond[:, T_i - 1] - discountBond[:, T - 1]
    ) / np.sum(discountBond[:, T_i - 1:], axis=1)

    return discountBond, par_swap_rate

def monte_carlo_swaption_pricing(numSim, f0, strike_rates, option_maturities, T, sigma, dT):

    f_t_num = np.zeros((numSim, T)) 
    f_t_num[:, 0] = f0  

    Wt = np.random.normal(0, np.sqrt(dT), (numSim, T))

    for i in range(1, T):
        drift_Updated = -sigma**2 * (T - i) / 2
        f_t_num[:, i] = (
            f_t_num[:, i - 1]
            + drift_Updated * dT
            + sigma * Wt[:, i]
        )
    f_t_num = np.maximum(f_t_num, 0)

    # Calculate initial discount factors for all maturities
    discountBond_init = np.array([1 / ((1 + f0) ** i) for i in range(1, T + 1)])

    # Monte Carlo pricing of swaptions
    swaptionPrices = []
    for K in strike_rates:
        pricesStrike = []
        
        #Simulation of all params
        for T_i in option_maturities:

            discountBond, par_swap_rate = simulatedParams(f_t_num, T_i, T)

            swaptionPayoffs = (np.maximum(par_swap_rate - K, 0) * np.sum(discountBond[:, T_i - 1:], axis=1)
                               / discountBond_init[T_i - 1])

            swaptionAVG = np.mean(swaptionPayoffs) * discountBond_init[T_i - 1]

            pricesStrike.append(swaptionAVG)
        swaptionPrices.append(pricesStrike) 

    mc_swap = pd.DataFrame(
        swaptionPrices,
        index=[f"{int(K * 100)}%" for K in strike_rates],
        columns=[f"T{i}" for i in option_maturities]
    )
    mc_swap.index.name = "Strike Rate"
    mc_swap.columns.name = "Option Maturity"

    return mc_swap


# Parameters
numSim = 1000000  # Number of Monte Carlo simulations
f0 = 0.03  # Initial LIBOR rate
strike_rates = np.arange(0.01, 0.06, 0.01)  # Strike rates
option_maturities = np.arange(1, 10)  # Option maturities (in years)
T = 10  # Final maturity (in years)
sigma = 0.01  # Volatility
dT = 1  # Time step (in years)

monte_carlo_swaption_pricing(numSim, f0, strike_rates, option_maturities, T, sigma, dT)


Option Maturity,T1,T2,T3,T4,T5,T6,T7,T8,T9
Strike Rate,,,,,,,,,
1%,0.151017,0.137100,0.122796,0.108077,0.092976,0.077527,0.061676,0.045179,0.027190
2%,0.099766,0.093188,0.085522,0.076929,0.067521,0.057374,0.046469,0.034584,0.020886
3%,0.060453,0.058999,0.056133,0.052070,0.046966,0.040922,0.033922,0.025766,0.015644
4%,0.033037,0.034401,0.034434,0.033317,0.031163,0.028044,0.023941,0.018642,0.011407
5%,0.015984,0.018253,0.019579,0.020027,0.019626,0.018395,0.016289,0.013072,0.008081
